In [1]:
from model import MusicTransformer
from custom.layers import *
from custom import callback
from tensorflow.python import keras
import params as par
from tensorflow.python import enable_eager_execution
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
tf.executing_eagerly()

True

Ściągnięcie zbiorów uczących (skrypt linuxowy, można też samemy podać jakiś inny zbiór):  
sh dataset/script/{ecomp_piano_downloader, midi_world_downloader, ...}.sh  
(np. sh dataset/script/classic_piano_dataset.sh)  
  
Preprocessing (pliki muzyczne są konwertowane do sekwencji liczb i zapisywane za pomocą pickle):  
python preprocess.py {midi_load_dir} {dataset_save_dir}  
(np. python preprocess.py "dataset\scripts\classic_piano_dataset"   "dataset\scripts\classic_piano_dataset_preprocessed")  
  
(jeśli skrypt chce się uruchomić prosto z notebooka to wystarczy przed komendą shellową wpisać "!")  

Uczenie:  
  
python train.py --epochs={NUM_EPOCHS} --load_path={NONE_OR_LOADING_DIR} --save_path={SAVING_DIR} --max_seq={SEQ_LENGTH} --pickle_dir={DATA_PATH} --batch_size={BATCH_SIZE} --l_r={LEARNING_RATE}  
  
(Przykładowo:  
python train.py --epochs=300 --save_path="sample_model"   
--max_seq=200 --pickle_dir="dataset\scripts\classic_piano_dataset_preprocessed" --batch_size=4)  

W pliku "sample_model" znajduje się wstępnie nauczona przeze mnie sieć nauronowa (max_seq=500, batch=3, epochs=400 ~= 4 godziny uczenia). Aby sprawdzić jej działanie:

In [2]:
from model import MusicTransformerDecoder
opt = Adam(par.l_r)
mt = MusicTransformerDecoder(
  	embedding_dim=256, vocab_size=par.vocab_size, 
  	num_layer=6, 
  	max_seq=500,
  	dropout=0.1,
  	debug=False,
    loader_path = r"sample_model"
)
# mt.compile(optimizer=opt, loss=callback.TransformerLoss())

In [5]:
output = mt.generate(prior=[70])
output

array([ 70, 370,  65, 367,  41, 367,  48, 321, 179, 191, 195, 336, 185,
       191, 176, 371,  71, 368,  68, 368,  63, 368,  59, 366,  32, 366,
        44, 352, 199, 368,  72, 314, 172, 369,  77, 367,  67, 256, 200,
       302, 205, 195, 369,  76,  67, 259, 195, 304, 196, 191, 187, 205,
       369,  77, 367,  68, 367,  65, 366,  37, 366,  49, 321, 369,  76,
       324, 368,  77, 288, 205, 369,  76, 323, 204, 368,  76, 325, 205,
       196, 193, 165, 177, 204, 204, 368,  66, 366,  39, 367,  51, 369,
        75, 322, 194, 167, 179, 203, 368,  77, 271, 205, 366,  76, 263,
       204, 302, 167, 179, 365,  68, 362,  61, 363,  64, 362,  53, 362,
        56, 321, 366,  73, 321, 366,  76, 324, 367,  77, 289, 365,  80,
       289, 196, 189, 192, 181, 184, 361,  56, 361,  61, 360,  53, 360,
        56, 361,  68, 365,  73, 256, 205, 208, 321, 184, 189, 181, 196,
       201, 361,  61, 360,  53, 360,  56, 365,  73, 363,  68, 325, 367,
        77, 290, 365,  80, 290, 184, 361,  56, 361,  60, 360,  5

In [6]:
from  midi_processor.processor import decode_midi
decode_midi(output.tolist(), 'generated_2.mid')

info removed pitch: 51
info removed pitch: 63
info removed pitch: 67
info removed pitch: 57
info removed pitch: 63


In [ ]:
Jeśli chcesz przesłuchać stworzonej piosenki a nie posiadasz np. DAW (FL Studio), można użyć sekwencera online:
https://onlinesequencer.net/import
Uploaduje się piosenkę (.mid), akceptuje domyśle ustawienia i na ekranie pokazuje się widok piano roll (można też odtworzyć utwór).